In [ ]:
# Install pylibCZIrw and dependencies
! pip install --upgrade pip
! pip install "pylibCZIrw~=4.1" "cztile>=0.0,<1.0" matplotlib tqdm pooch

## pylibCZIrw - Tutorial

In this notebook one can see how to read and write CZI (Carl Zeiss Image) files using from Python using **[pylibCZIrw](https://pypi.org/project/pylibczirw/)**. This package is wrapper for the **libCZIrw** (C++) library and can be used both in Windows and Linux.

### Remarks

* At the moment, [pylibCZIrw](https://pypi.org/project/pylibczirw/) completely abstracts away the subblock concept, both in the reading and in the writing APIs.
* If [pylibCZIrw](https://pypi.org/project/pylibczirw/) is extended in the future to support subblock-based access (e.g. accessing acquisition tiles), this API must not be altered.

### Disclaimer

This notebook is free to use for everybody. Carl Zeiss Microscopy GmbH's ZEN software undertakes no warranty concerning the use of those scripts, image analysis settings and ZEN experiments. Use them on your own risk.

Additionally Carl Zeiss Microscopy GmbH's ZEN software allows connection and usage to the third party software packages.
Therefore Carl Zeiss Microscopy GmbH undertakes no warranty concerning those software packages, makes no representation that they will work on your system and/or hardware and will not be liable for any damages caused by the use of this extension. **By using any of those examples you agree to this disclaimer.**

Version: 2024.01.22

Copyright (c) 2024 Carl Zeiss AG, Germany. All Rights Reserved.

### Required Module Imports

Both reading and writing APIs are contained within the **czi** object in pylibCZIrw, so let's start by importing it.

Additional modules are imported alongside the **pylibCZIrw** package.

In [ ]:
from pylibCZIrw import czi as pyczi
from pylibCZIrw.czi import ReaderFileInputTypes, CacheOptions, CacheType
import json
import timeit
from matplotlib import pyplot as plt
import matplotlib.cm as cm
import numpy as np
import os, sys
from tqdm import tqdm
from tqdm.contrib import itertools as it
from matplotlib.patches import Rectangle
from IPython.display import display, JSON

# show the used python env
print("Using:", sys.executable)

## Define parameters for data loading

In [ ]:
# Define CZI locations

czifile_DAPI_PGC = r"https://github.com/zeiss-microscopy/OAD/raw/master/jupyter_notebooks/pylibCZIrw/data/DAPI_PGC.czi"

czifile_5dstack = r"https://github.com/zeiss-microscopy/OAD/raw/master/jupyter_notebooks/pylibCZIrw/data/T%3D3_Z%3D5_CH%3D2_X%3D240_Y%3D170.czi"

czifile_scenes = r"https://github.com/zeiss-microscopy/OAD/raw/master/jupyter_notebooks/pylibCZIrw/data/w96_A1%2BA2.czi"

czifile_timeseries = r"https://github.com/zeiss-microscopy/OAD/blob/master/jupyter_notebooks/pylibCZIrw/data/CellDivision_T%3D16_CH%3D2_ZSTD.czi"

### Opening a CZI (read-only)

A CZI file can be opened from disk in a context manager or alternatively it can also be opened directly from a seekable http url.

* `pyczi.ReaderFileInputTypes.Curl` reads from http
* * `pyczi.ReaderFileInputTypes.Standard` reads from file directly (default)

```python
from pylibCZIrw import pyczi

with czi.open_czi(file_path) as czi:

    # add your code here

with czi.open_czi(file_url, pyczi.ReaderFileInputTypes.Curl) as czi:

    # add your code here
```

Remark: Inside this notebook only the option to read from http is used which removes the need to download the data when using Colab notebooks.

### Open a CZI and read metadata

We can open a czi using a context manager and then read the full metadata either as a raw XML string or as a dictionary

In [ ]:
# open the CZI for reading using a context manager (preferred way to do it)
with pyczi.open_czi(czifile_5dstack, pyczi.ReaderFileInputTypes.Curl) as czidoc:
    # get the raw metadata as XML
    md_xml = czidoc.raw_metadata
    print(md_xml[:1000])

In [ ]:
# open the CZI for reading
with pyczi.open_czi(czifile_scenes, pyczi.ReaderFileInputTypes.Curl) as czidoc:
    # get the raw metadata as a dictionary
    md_dict = czidoc.metadata

    # show some parts of it
    # print(json.dumps(md_dict["ImageDocument"]["Metadata"]["Information"]["Image"], sort_keys=False, indent=4))
    display(JSON(md_dict["ImageDocument"]["Metadata"]["Information"]["Image"], expanded=False))

## Get Image dimensions
There are different properties that allow us to retrieve information about dimensions.
The **`total_bounding_box`** gives us all the dimensions of all **orthogonal planes** of the CZI image file.

In [ ]:
with pyczi.open_czi(czifile_5dstack, pyczi.ReaderFileInputTypes.Curl) as czidoc:
    # get the image dimensions as an dictionary, where the key identifies the dimension
    total_bounding_box = czidoc.total_bounding_box

print(total_bounding_box)

The **`total_bounding_rectangle`** gives us the X and Y dimensions of the CZI, i.e. the (X, Y) of the **`total_bounding_box`**.

In [ ]:
with pyczi.open_czi(czifile_scenes, pyczi.ReaderFileInputTypes.Curl) as czidoc:
    # get the total bounding box for all scenes
    total_bounding_rectangle = czidoc.total_bounding_rectangle

print(total_bounding_rectangle)

**Scenes are not orthogonal to the other dimensions.**
They are contained within the 2D planes and can be seen simply as tags for certain regions.

The **`scene_bounding_rectangle`** give us the bounding rectangle for each scene.

In [ ]:
with pyczi.open_czi(czifile_scenes, pyczi.ReaderFileInputTypes.Curl) as czidoc:
    # get the bounding boxes for each individual scene
    scenes_bounding_rectangle = czidoc.scenes_bounding_rectangle

print(scenes_bounding_rectangle)

## Get the pixel type
A channel's pixel type can be discovered with using: **`get_channel_pixel_type`**

In [ ]:
with pyczi.open_czi(czifile_scenes, pyczi.ReaderFileInputTypes.Curl) as czidoc:
    # get the pixel type for the 1st channel
    c0_pixel_type = czidoc.get_channel_pixel_type(0)

print(c0_pixel_type)

Or we can simply get all pixel types using: **`pixel_types`**

In [ ]:
with pyczi.open_czi(czifile_scenes, pyczi.ReaderFileInputTypes.Curl) as czidoc:
    # get all pixel types as a dictionary, where the key is the channel index
    pixel_type = czidoc.pixel_types

print(pixel_type)

## Read pixel data
Pixel data is read and returned as a numpy arry. The shape of the returned array and its data type will depend on the specified `pixel_type`.
- BGR pixel type -> [y, x, 3]
- Gray pixel type -> [y, x, 1]

The background_pixel determines the pixel value for the background (i.e. regions where no pixel data exists),
 and must be consistent with the `pixel_type`

|pixel_type | background_pixel type | Default value  | 
--- | --- | ---
|BGR|3-value tuple |(0, 0, 0)|
|Gray|Scalar value bounded by the gray scale|0|

### Read whole 2D plane

In order to **`read`** pixel data from a CZI image file it is important to understand the concept of "planewise reading".

Reading data planewise means that every read operation yields a 2D array with the shape (Y, X, 1) or (Y, X, 3) depending on the requested pixel type.
We can specify which 2D plane to read using the **`plane`** argument (dictionary).

* **`plane = {'C': 0, 'Z': 2, 'T': 1}`** - this will read a 2D plane from C=0, Z=2 and T=1
* if **`plane`** is not specified at all the default is to use the 1st index for all dimensions

In order to read a complete Z-Stack, one would have to read each of the Z-slices using the plane argument. This can be easily created by "wrapping" those calls into some utility functions.
To get started let's read a complete 2D plane.

In [ ]:
with pyczi.open_czi(czifile_5dstack, pyczi.ReaderFileInputTypes.Curl) as czidoc:
    # define some plane coordinates
    plane_1 = {'C': 0, 'Z': 2, 'T': 1}
    plane_2 = {'C': 1, 'Z': 3, 'T': 2}

    # equivalent to reading {'C': 0, 'Z': 0, 'T': 0}
    frame_0 = czidoc.read()

    # get the shape of the 2d plane - the last dime indicates the pixel type
    # 3 = BGR and 1 = Gray
    print("Array Shape: ", frame_0.shape)

    # get specific planes 
    frame_1 = czidoc.read(plane=plane_1)
    frame_2 = czidoc.read(plane=plane_2)

fig, ax = plt.subplots(1, 3, figsize=(15, 5))
ax[0].imshow(frame_0[..., 0], cmap=cm.inferno)
ax[0].set_title("Frame_0")
ax[1].imshow(frame_1[..., 0], cmap=cm.inferno)
ax[1].set_title("Frame_1")
ax[2].imshow(frame_2[..., 0], cmap=cm.Greens_r)
ax[2].set_title("Frame_2")

### Read ROI inside a specific 2D plane

We can also read an ROI within the 2D plane by using the **`roi`** argument.
This is particularly useful when dealing with large images whose planes would not fit into memory.

The **`roi`** is a tuple defined as a (axis-aligned) rectangle in **(x, y, w, h)** form, with:

* x = x coordinate of upper-left point
* y = y coordinate of upper-left point
* w = rectangle width
* h = rectangle height

In [ ]:
# define the ROI using upper left, widht and height as coordinates
my_roi = (200, 400, 800, 600)

with pyczi.open_czi(czifile_scenes, pyczi.ReaderFileInputTypes.Curl) as czidoc:
    ch0 = czidoc.read(roi=my_roi, plane={'C': 0})
    ch1 = czidoc.read(roi=my_roi, plane={'C': 1})

fig, ax = plt.subplots(1, 2, figsize=(10, 5))
ax[0].imshow(ch0[..., 0], cmap=cm.inferno, vmin=100, vmax=4000)
ax[0].set_title("ch0")
ax[1].imshow(ch1[..., 0], cmap=cm.Greens_r, vmin=100, vmax=4000)
ax[1].set_title("ch1")

### Read ROI only for one scene
We can also get pixel data exclusively from one scene using the **`scene`** argument. This argument acts as
a filter returning only data from the specified scene. If not set, data from all scenes is returned.

In [ ]:
with pyczi.open_czi(czifile_scenes, pyczi.ReaderFileInputTypes.Curl) as czidoc:
    # read a 2D image from a specific channel and scene
    c0_s0 = czidoc.read(plane={'C': 0}, scene=0)
    c1_s0 = czidoc.read(plane={'C': 1}, scene=0)
    c0_s1 = czidoc.read(plane={'C': 0}, scene=1)
    c1_s1 = czidoc.read(plane={'C': 1}, scene=1)

fig, ax = plt.subplots(2, 2, figsize=(16, 16))
ax[0, 0].imshow(c0_s0[..., 0], cmap=cm.inferno, vmin=100, vmax=4000)
ax[0, 0].set_title("scene:0 ch:0")
ax[0, 1].imshow(c1_s0[..., 0], cmap=cm.Greens_r, vmin=100, vmax=4000)
ax[0, 1].set_title("scene:0 ch:1")
ax[1, 0].imshow(c0_s1[..., 0], cmap=cm.inferno, vmin=100, vmax=4000)
ax[1, 0].set_title("scene:1 ch:0")
ax[1, 1].imshow(c1_s1[..., 0], cmap=cm.Greens_r, vmin=100, vmax=4000)
ax[1, 1].set_title("scene:1 ch:0")

### Additional parameters

We can specify the **`pixel_type`** in which we want the data to be read. If the desired pixel type is not specified, the data is returned in its original pixel format.

In [ ]:
with pyczi.open_czi(czifile_5dstack, pyczi.ReaderFileInputTypes.Curl) as czidoc:
    
    # get image pixel and specify the desired pixel type
    frame_gray8 = czidoc.read(pixel_type='Gray8')
    frame_gray32float = czidoc.read(pixel_type='Gray32Float')
    frame_bgr48 = czidoc.read(pixel_type='Bgr48')

print('shape {0}, data type {1}'.format(frame_gray8.shape, frame_gray8.dtype))
print('shape {0}, data type {1}'.format(frame_gray32float.shape, frame_gray32float.dtype))
print('shape {0}, data type {1}'.format(frame_bgr48.shape, frame_bgr48.dtype))

We can also specify a **`zoom`** factor between 0 and 1. This can be very useful to read images at a lower resolution,
because not every application requires the full resolution.

In [ ]:
# define the ROI using upper left, width and height as coordinates
my_roi = (300, 400, 300, 300)

# define two different zoom factors
zl1 = 1.0
zl2 = 0.25

with pyczi.open_czi(czifile_scenes, pyczi.ReaderFileInputTypes.Curl) as czidoc:
    
    z1 = czidoc.read(plane={'C': 0}, roi=my_roi, scene=0, zoom=zl1, pixel_type='Gray8')
    z2 = czidoc.read(plane={'C': 0}, roi=my_roi, scene=0, zoom=zl2, pixel_type='Gray8')

fig, ax = plt.subplots(1, 2, figsize=(16, 8))
ax[0].imshow(z1[..., 0], cmap=cm.inferno)
ax[1].imshow(z2[..., 0], cmap=cm.inferno)

print("Shape Zoomlevel = ", zl1, z1.shape)
print("Shape Zoomlevel = ", zl2, z2.shape)

## Writing a CZI file

* **The writing API only allows creating new CZI files.**
* editing existing czi file is not supported.
* writing API is follows the same principles as the reading API.

### Create a new CZI

Similarly to reading, before writing, we must create a CZI instance.
Ideally this is done using a context manager again.

### Write pixel data

Writing pixel data is very similar to reading albeit with fewer arguments:

* the **`plane`** to which the pixel data should be written.
* the data to be written, as a numpy array.

The pixel type in which the data is written is automatically determined from the shape and data type of the provided array.

In [ ]:
array = np.random.randint(low=1, high=10, size=(32, 64, 64)).astype(np.uint8)
print(array.shape)

In order to create a new CZI file we must provide the file name.
The optional flag (**`exist_ok`**) allows overwriting if set to **`True`** (default is **`False`**).

In [ ]:
# create the filename for the new CZI image file
newczi_4dstack = os.path.join(os.getcwd(), "newCZI_stack.czi")
numZ = 32

# open a new CZI and allow overwrite (!!!) to play around ...
with pyczi.create_czi(newczi_4dstack, exist_ok=True) as czidoc_w:
    # loop over all z-planes and channels
    for frame in tqdm(range(numZ)):
        # get the 2d array for the current plane and add axis to get (Y, X, 1) as shape
        # write the plane with shape (Y, X, 1) to the new CZI file
        czidoc_w.write(
            data=array[frame, ...][..., np.newaxis],
            plane={"Z": frame}
        )

We can also write data to a specific **`location`** within the plane by providing the upper left pixel coordinates.

To illustrate this idea let's write all z-planes from the image "side-by-side" into a new CZI image.

In [ ]:
# create the filename for the new CZI image file
newczi_zloc = os.path.join(os.getcwd(), "newCZI_zloc.czi")
xstart = 0

with pyczi.create_czi(newczi_zloc, exist_ok=True) as czidoc_w:
    # loop over all z-planes
    for frame in tqdm(range(4)):
        # get the 2d array for the current plane and add axis to get (Y, X, 1) as shape
        # for fun - write the z-planes to different locations as a single channel
        czidoc_w.write(
            data=array[frame, ...][..., np.newaxis],
            plane={"C": 0},
            location=(xstart, 0)
        )

        # change the x-position for the next round to write "side-by-side"
        xstart = xstart + 80

![newCZI_stack.czi - shown in ZEN blue](screenshots/new_czi_stack.png)

### Write scenes

Writing a scene is in fact "just" writing pixel data to a specific region within a plane and tagging it with a **`scene`** index.
To illustrate this we now will write the z-planes to different scenes at different locations.

In [ ]:
# first step is to create some kind of grid of locations
locx = []
locy = []
xystart = 0
offset = 128

# create the list for the 4x4 grid locations for the 16 planes 
for x, y in it.product(range(4), range(4)):
    locx.append(xystart + offset * x)
    locy.append(xystart + offset * y)

In [ ]:
# create the filename for the new CZI image file
newczi_zscenes = os.path.join(os.getcwd(), "newCZI_scenes.czi")

with pyczi.create_czi(newczi_zscenes, exist_ok=True) as czidoc_w:
    # loop over all z-planes
    for frame in tqdm(range(16)):
        # get the 2d array for the current plane and add axis to get (Y, X, 1) as shape
        # for "fun" - write the z-planes to different scenes using the locations we just created
        czidoc_w.write(
            data=array[frame, ...][..., np.newaxis],
            plane={"C": 0},
            scene=frame,
            location=(locx[frame], locy[frame])
        )

![new_czi_scenes - shown in ZEN blue](screenshots/new_czi_scenes.png)

### ZSTD Compression

#### Why we have used ZSTD Compression?
It is a fast compression technique, which provides the high compression ratios. We are handling these three modes:

1. uncompressed: - Default setting, the image would not be compressed.
2. zstd0 - You can specify the parameters of it like "explicitlevel" between -131072 and 22. <b>(e.g. "zstd0:ExplicitLevel=10")</b>
3. zstd1 - You can specify the parameters of it like "explicitlevel" between -131072 and 22 or "preprocess" - only have HiLoByteUnpack. This is only done for zstd1 and for pixel types gray16 and Bgr48. <b>(e.g. "zstd1:ExplicitLevel=10" or "zstd1:Preprocess=HiLoByteUnpack")</b>

To read more: https://facebook.github.io/zstd/

In [ ]:
# create some array to write to CZI
array_big = np.random.randint(low=1, high=50, size=(4096, 4096)).astype(np.uint16)

# create the filename for the new CZI image file
newczi_uncompressed = os.path.join(os.getcwd(), "newCZI_uncompressed.czi")

# create the filename for the new CZI image file
newczi_compressed = os.path.join(os.getcwd(), "newCZI_compressed.czi")

In [ ]:
with pyczi.create_czi(newczi_compressed, exist_ok=True) as czidoc_w:
    
    # The CZIWriter class has a compression parameter that allows to define a default compression for each write call.
    # The write call will the allow you to change the compression for a specific case by setting its compression parameter explicitly as here changing to the zstd0: 
    # writing the image with "zstd0" compression.
    czidoc_w.write(
        data=array_big,
        plane={"C": 0, "T": 0, "Z": 0},
        scene=0,
        compression_options = "zstd0:ExplicitLevel=10",
        location=(0,0)
    )

In [ ]:
# The compression option could be defined while creating an czi file and could still be overwritten 
# with each individual call to the write function.
with pyczi.create_czi(newczi_uncompressed, exist_ok=True, compression_options = "uncompressed:") as czidoc_w:
                
        czidoc_w.write(
            data=array_big,
            plane={"C": 0, "T": 0, "Z": 0},
            scene=0,
            location=(0,0)
        )

In [ ]:
size = os.path.getsize(newczi_uncompressed)/1024.0**2
print(f"Size of the uncompressed CZI file : {np.round(size, 2)} MB ")

size_comp = os.path.getsize(newczi_compressed)/1024.0**2
print(f"Size of the compressed CZI file 1 : {np.round(size_comp, 2)} MB ")

print(f"ZSTD Compression Ratio 1: {np.round(size/size_comp, 2)}")

### Write Metadata

**Metadata is automatically written**, however, we can write some additional metadata by explicitly calling
 **`write_metadata`** with the following **optional** parameters:

- **`document_name`** (in ZEN: "Title")
- **`channel_names`** (in ZEN: "Channel Names")
- **`custom_attributes`** (Not visible in ZEN)
- **`scale_x`**, **`scale_y`**, **`scale_z`** in **meters (!)** (in ZEN: "Scaling (per Pixel)")

The parameter **`custom_attributes`** allows customized attributes with type int, float, boolean or string. If an attribute already exists, writing the same attribute again will replace the existing value.

This list of parameters will be extended on a case-by-case basis in the future.

***

In [ ]:
# create the filename for the new CZI image file
newczi_mdata = os.path.join(os.getcwd(), "newczi_mdata.czi")
numZ = 32

# open a new CZI and allow overwrite (!!!) to play around ...
with pyczi.create_czi(newczi_mdata, compression_options = "zstd0:ExplicitLevel=10", exist_ok=True) as czidoc_w:
    # loop over all z-planes
    for z in tqdm(range(numZ)):
        # get the 2d array for the current plane and add axis to get (Y, X, 1) as shape
        # write the plane with shape (Y, X, 1) to the new CZI file
        czidoc_w.write(
            data=array[z, ...][..., np.newaxis],
            plane={"Z": z}
        )

    # write the document title, channel names, custom attributes and XYZ scaling to the CZI file
    czidoc_w.write_metadata(
        document_name='new_czi_z=32',
        channel_names={0: "Red"},
        custom_attributes={"key1": "value1", "key2": "value2"},
        scale_x=0.1 * 10 ** -6,
        scale_y=0.1 * 10 ** -6,
        scale_z=0.2 * 10 ** -6
    )

![new_czi_z=32_metadata - shown in ZEN blue](screenshots/new_czi_z32_metadata.png)

## Subblock caching

In certain scenarios (e.g. reading from a stream or reading compressed images) caching uncompressed subblocks might yield a higher reading performance. A CZI reader allows for specifying caching options that lead to subblocks being cached after their first use.

In [ ]:
num_executions = 100

for use_cache in [True, False]:
    cache_options = CacheOptions(CacheType.Standard) if use_cache else None

    with pyczi.open_czi(newczi_compressed, ReaderFileInputTypes.Standard, cache_options=cache_options) as czi_document:
        total_time = timeit.timeit(czi_document.read, number=num_executions)
        average_time = total_time / num_executions

        print(
            f"Average time for read_czi (use_cache={use_cache}) over "
            f"{num_executions} executions: {average_time:.6f} seconds"
        )

## Tiling a large image

The read and write API allow the user to work with large images by allowing breaking down the access using the **`roi`** argument.
However, defining these rois (aka tiles) can be a cumbersome process.

The **czitile** package was created to answer this need.

**The package contains pure logic for tiling a rectangle. This means it is not file or format bound in any way.**

Let's read the original file using cztile and write a new with the same dimensions and scenes but with
random data instead.

### Define a 2D processing function

The benefits of tiling are evident when the data needs to be broken down for processing.
Segmentation using Deep Neural Networks is a great example of a task where images must be cut to fit.
We can use **czitile** to break down the image into tiles with some overlap.

We can illustrate this with a simpler 2D processing function first.

In [ ]:
from skimage.filters import gaussian


# define a generic 2D processing function - could by a DL segmentation as well
def process2d(image2d: np.ndarray, **kwargs: int) -> np.ndarray:
    # insert or modify the desired processing function here
    image2d = gaussian(
        image2d, sigma=kwargs["sigma"],
        preserve_range=True,
        mode='nearest'
    ).astype(image2d.dtype)

    return image2d

In [ ]:
# import the "tiling strategy" from the cztile package
from cztile.fixed_total_area_strategy import AlmostEqualBorderFixedTotalAreaStrategy2D

# create the filename for the new CZI image file
newczi_tile = os.path.join(os.getcwd(), "newczi_tilewise.czi")

# create a "tile" by specifying the desired tile dimension and the
# minimum required overlap between tiles (depends on the processing)
tiler = AlmostEqualBorderFixedTotalAreaStrategy2D(
    total_tile_width=1600,
    total_tile_height=1400,
    min_border_width=128
)

# create CZI instance to read some metadata 
with pyczi.open_czi(czifile_DAPI_PGC, ReaderFileInputTypes.Curl) as czidoc_r:
    # get the size of the bounding rectange for the scence
    tiles = tiler.tile_rectangle(czidoc_r.scenes_bounding_rectangle[0])

# show the created tile locations
for tile in tiles:
    print(tile.roi.x, tile.roi.y, tile.roi.w, tile.roi.h)

### Opening a CZI file for reading and write to a new CZI in parallel

The idea of tile-wise processing can enable the following approach:

* open the "original CZI image file for reading
* tile the image using **czitile**
* use the computed tiles as the **roi** parameters to read a "tile" of data
* apply the desired processing to the read tile
* write the processed tile into the new CZI at the correct location using the tile properties

In [ ]:
# open an existing CZI file for reading planes and a new CZI to insert
# the newly processed chunks into the new CZI file

# create to figures
fig1, ax1 = plt.subplots(1, 4, figsize=(16, 8))
fig2, ax2 = plt.subplots(1, 4, figsize=(16, 8))

# counter figures
a = 0

# open a CZI instance to read and in parallel one to write
with pyczi.open_czi(czifile_DAPI_PGC, ReaderFileInputTypes.Curl) as czidoc_r:
    with pyczi.create_czi(newczi_tile, exist_ok=True) as czidoc_w:
        # loop over all tiles created by the "tiler"
        for tile in tqdm(tiles):
            # read a specific tile from the CZI using the roi parameter
            tile2d = czidoc_r.read(
                plane={"C": 0},
                roi=(
                    tile.roi.x,
                    tile.roi.y,
                    tile.roi.w,
                    tile.roi.h
                )
            )

            # process the current tile using a function
            tile2d_processed = process2d(tile2d, sigma=11)

            # show sthe tiles to illustrate the idea
            ax1[a].imshow(tile2d[..., 0], interpolation="nearest", cmap="gray", vmin=100, vmax=4000)
            ax1[a].set_title("Tile: " + str(a))
            ax2[a].imshow(tile2d_processed[..., 0], interpolation="nearest", cmap="gray", vmin=100, vmax=4000)
            ax2[a].set_title("Tile: " + str(a))

            # write the new CZI file using the processed data
            czidoc_w.write(
                tile2d_processed,
                plane={"C": 0},
                location=(
                    tile.roi.x,
                    tile.roi.y
                )
            )

            a += 1

In [ ]:
import random

# read the CZI pixel data
with pyczi.open_czi(czifile_DAPI_PGC, ReaderFileInputTypes.Curl) as czidoc_r:
    data_orig = czidoc_r.read(plane={"C": 0})

with pyczi.open_czi(newczi_tile) as czidoc_r:
    data_processed = czidoc_r.read(plane={"C": 0})

# show the results
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8))
ax1.imshow(data_orig[..., 0], interpolation="nearest", cmap="gray")
ax1.set_title('Original CZI')
ax2.imshow(data_processed[..., 0], interpolation="nearest", cmap="gray")
ax2.set_title('Processed CZI - TileWise')

# define the tile to be shown
tn = 0

# add rectangle to plot to visualize the tile
ax1.add_patch(
    Rectangle((tiles[tn].roi.x, tiles[tn].roi.y), tiles[tn].roi.w, tiles[tn].roi.h,
              edgecolor="red",
              fill=False,
              lw=3,
              ls="--")
)

ax2.add_patch(
    Rectangle((tiles[tn].roi.x, tiles[tn].roi.y), tiles[tn].roi.w, tiles[tn].roi.h,
              edgecolor="red",
              fill=False,
              lw=3,
              ls="--")
)

# show the results
fig, (ax3, ax4) = plt.subplots(1, 2, figsize=(16, 8))
ax3.imshow(data_orig[..., 0], interpolation="nearest", cmap="gray")
ax3.set_title('Original CZI')
ax4.imshow(data_processed[..., 0], interpolation="nearest", cmap="gray")
ax4.set_title('Processed CZI- TileWise')

edge_color_opacity = 1.0
face_color_opacity = 0.6

# visualize a single tile
for tile in tiles:
    # use random colors to visualize the tiles
    r = random.random()
    b = random.random()
    g = random.random()
    edgecolor = (r, g, b, face_color_opacity)
    facecolor = (r, g, b, face_color_opacity)

    # add rectangle to plot
    ax3.add_patch(
        Rectangle((tile.roi.x, tile.roi.y), tile.roi.w, tile.roi.h,
                  edgecolor=edgecolor,
                  fill=False,
                  lw=3,
                  ls="--")
    )

    ax4.add_patch(
        Rectangle((tile.roi.x, tile.roi.y), tile.roi.w, tile.roi.h,
                  facecolor=facecolor,
                  fill=True,
                  lw=3,
                  ls="--")
    )

    c = + 1